##### Module import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성

In [2]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

##### Data import

In [3]:
print(os.getcwd())

c:\Users\Owner\Desktop\윤태준\고용패널\고용패널 파이썬 분석\2023_04_06_변수추가\변수추가


In [4]:
df = pd.read_csv('2023_04_10_data_step.csv', encoding='cp949')

In [5]:
df_1 = df.copy()
df_1.columns

Index(['pid', 'majorcat', 'province', 'area', 'sex', 'age', 'a003', 'a010',
       'a038', 'a039',
       ...
       'big_companyG', 'work_satiG', 'company_satiG', 'supply', 'insurance',
       '주당근로일', '주당근로시간', '직장에서의복리후생제공여부', '보험가입여부', '직장만족도'],
      dtype='object', length=106)

##### 다변량 모델

#### 복지 모델

In [6]:
model_1 = sm.Logit.from_formula('''LongWork2 ~ C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_1.summary()

Optimization terminated successfully.
         Current function value: 0.544362
         Iterations 5
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5805
Method:                           MLE   Df Model:                            2
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                 0.06165
Time:                        20:19:01   Log-Likelihood:                -3161.7
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 6.229e-91
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -1.6740      0.055    -30.503      0.000      -1.782      -1.566
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     1.0717      0.069     15.474      0.000       0.936       1.207
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.6123      0.067      9.104      0.000       0.481       0.744
==================================================================================================================
"""

In [7]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_1 = pd.DataFrame(
    {
        "OR": model_1.params,
        "Lower CI": model_1.conf_int()[0],
        "Upper CI": model_1.conf_int()[1],
    }
)
odds_ratios_1 = np.exp(odds_ratios_1)
odds_ratios_1

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.187490,0.168370,0.208782
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.920222,2.549571,3.344757
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.844752,1.616896,2.104717


In [8]:
model_2 = sm.Logit.from_formula('''LongWork2 ~ + C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    +  C(보험가입여부,Treatment('가입하지않음')) + 주당근로일 ''', df_1).fit()
print('==================================Model Summary=========================================')
model_2.summary()

Optimization terminated successfully.
         Current function value: 0.542397
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5804
Method:                           MLE   Df Model:                            3
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                 0.06503
Time:                        20:19:20   Log-Likelihood:                -3150.2
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 1.154e-94
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -2.4654      0.183    -13.468      0.000      -2.824      -2.107
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     1.0545      0.069     15.230      0.000       0.919       1.190
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.5193      0.070      7.468      0.000       0.383       0.656
주당근로일                                              0.1781      0.038      4.636      0.000       0.103       0.253
==================================================================================================================
"""

In [9]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_2 = pd.DataFrame(
    {
        "OR": model_2.params,
        "Lower CI": model_2.conf_int()[0],
        "Upper CI": model_2.conf_int()[1],
    }
)
odds_ratios_2 = np.exp(odds_ratios_2)
odds_ratios_2

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.084973,0.059356,0.121645
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.870668,2.506366,3.287921
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.680887,1.466717,1.926329
주당근로일,1.194893,1.108240,1.288322


In [10]:
model_3 = sm.Logit.from_formula('''LongWork2 ~ 
    + C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))
    + 주당근로일 + C(직장만족도,Treatment('그렇지않다'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_3.summary()

Optimization terminated successfully.
         Current function value: 0.524463
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5802
Method:                           MLE   Df Model:                            5
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                 0.09595
Time:                        20:20:32   Log-Likelihood:                -3046.1
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                1.753e-137
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -3.4564      0.208    -16.637      0.000      -3.864      -3.049
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     0.9262      0.071     13.046      0.000       0.787       1.065
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.5731      0.071      8.095      0.000       0.434       0.712
C(직장만족도, Treatment('그렇지않다'))[T.그렇다]                1.2023      0.094     12.795      0.000       1.018       1.386
C(직장만족도, Treatment('그렇지않다'))[T.보통이다]               0.5473      0.098      5.565      0.000       0.355       0.740
주당근로일                                              0.2268      0.039      5.790      0.000       0.150       0.304
==================================================================================================================
"""

In [11]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3 = pd.DataFrame(
    {
        "OR": model_3.params,
        "Lower CI": model_3.conf_int()[0],
        "Upper CI": model_3.conf_int()[1],
    }
)
odds_ratios_3 = np.exp(odds_ratios_3)
odds_ratios_3

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.031544,0.020993,0.047397
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.524935,2.196945,2.901892
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.773803,1.543968,2.037851
"C(직장만족도, Treatment('그렇지않다'))[T.그렇다]",3.327845,2.768079,4.000808
"C(직장만족도, Treatment('그렇지않다'))[T.보통이다]",1.728622,1.425571,2.096097
주당근로일,1.254535,1.161838,1.354628
